In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp
import pickle
from tqdm import tqdm
import os

In [6]:
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]
num_images_per_category = 1000
images = []
labels = []

for label, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    count = 0
    for filename in os.listdir(category_path):
        if count >= num_images_per_category:
            break
        try:
            img_path = os.path.join(category_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (64, 64))  # Resize to standard size
            images.append(img)
            labels.append(label)
            count += 1
        except:
            continue  # Ignore unreadable files
images = np.array(images)
labels = np.array(labels)
print(f"Loaded {len(images)} images with shape {images[0].shape}")


Loaded 2000 images with shape (64, 64)


In [9]:
def extract_hybrid_features(img, pca_components=32):
    # --- Classical Features: ORB ---
    orb = cv2.ORB_create(nfeatures=64)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    if descriptors is None:
        descriptors = np.zeros((64, 32), dtype=np.uint8)
    elif descriptors.shape[0] < 64:
        descriptors = np.vstack([descriptors, np.zeros((64 - descriptors.shape[0], 32), dtype=np.uint8)])
    else:
        descriptors = descriptors[:64]
    orb_features = descriptors.flatten()  # Shape: 64 x 32 = 2048

    # --- Quantum-aware: Flatten image for amplitude embedding ---
    img_norm = img / 255.0
    flat = img_norm.flatten()
    if len(flat) < 256:
        padded = np.pad(flat, (0, 256 - len(flat)))
    else:
        padded = flat[:256]
    amp_features = padded  # For amplitude encoding (real-valued)

    # --- PCA Projection for QAOA Encoding Sim ---
    pca = PCA(n_components=pca_components)
    flattened_img = img.flatten().reshape(1, -1)  # shape (1, 4096) or so
    pca_data = pca.fit_transform(flattened_img).flatten()


    # --- Combine all ---
    hybrid_feature_vector = np.concatenate((orb_features, amp_features, pca_data))
    return hybrid_feature_vector


In [10]:
hybrid_features = []
for img in tqdm(images, desc="Extracting hybrid features"):
    feats = extract_hybrid_features(img)
    hybrid_features.append(feats)

hybrid_features = np.array(hybrid_features)
print("Hybrid feature shape:", hybrid_features.shape)


Extracting hybrid features:   0%|                                                             | 0/2000 [00:00<?, ?it/s]


ValueError: n_components=32 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(hybrid_features)
y = labels

print("Feature vector normalized. Ready for QCNN input or classical classifier.")


In [ ]:
output_file = "hybrid_encoded_petimages_qcnn.npz"
np.savez_compressed(output_file, X=X, y=y)

print(f"Hybrid encoded features saved to: {output_file}")

In [ ]:
# Load data
data = np.load("hybrid_encoded_petimages_qcnn.npz")
X_loaded = data['X']
y_loaded = data['y']

# Print first 5 examples
for i in range(5):
    print(f"Label: {y_loaded[i]}, Encoded Feature: {X_loaded[i]}")


In [11]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# --- Load Dataset ---
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]
num_images_per_category = 1000
images = []
labels = []

for label, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    count = 0
    for filename in os.listdir(category_path):
        if count >= num_images_per_category:
            break
        try:
            img_path = os.path.join(category_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)
            count += 1
        except Exception as e:
            continue

images = np.array(images)
labels = np.array(labels)
print(f"Loaded {len(images)} images with shape {images[0].shape}")

# --- Hybrid Feature Extraction ---
def extract_hybrid_features(img, pca_model=None, pca_components=32):
    orb = cv2.ORB_create(nfeatures=64)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    if descriptors is None:
        descriptors = np.zeros((64, 32), dtype=np.uint8)
    elif descriptors.shape[0] < 64:
        descriptors = np.vstack([descriptors, np.zeros((64 - descriptors.shape[0], 32), dtype=np.uint8)])
    else:
        descriptors = descriptors[:64]
    orb_features = descriptors.flatten()

    img_norm = img / 255.0
    flat = img_norm.flatten()
    if len(flat) < 256:
        padded = np.pad(flat, (0, 256 - len(flat)))
    else:
        padded = flat[:256]
    amp_features = padded

    flattened_img = img.flatten().reshape(1, -1)
    if pca_model is not None:
        pca_data = pca_model.transform(flattened_img).flatten()
    else:
        pca_data = np.zeros(pca_components)

    hybrid_feature_vector = np.concatenate((orb_features, amp_features, pca_data))
    return hybrid_feature_vector

# --- PCA Training ---
print("Fitting PCA model...")
flat_images = images.reshape(images.shape[0], -1)
pca_model = PCA(n_components=32)
pca_model.fit(flat_images)

# --- Feature Vector Construction ---
hybrid_features = []
for img in tqdm(images, desc="Extracting hybrid features"):
    feats = extract_hybrid_features(img, pca_model=pca_model, pca_components=32)
    hybrid_features.append(feats)

hybrid_features = np.array(hybrid_features)
print("Hybrid feature shape:", hybrid_features.shape)

# --- Standardize Features ---
scaler = StandardScaler()
X = scaler.fit_transform(hybrid_features)
y = labels
print("Feature vector normalized. Ready for QCNN input or classical classifier.")

# --- Save Features ---
output_file = "hybrid_encoded_petimages_qcnn.npz"
np.savez_compressed(output_file, X=X, y=y)
print(f"Hybrid encoded features saved to: {output_file}")

# --- Load & Verify ---
data = np.load(output_file)
X_loaded = data['X']
y_loaded = data['y']
for i in range(5):
    print(f"Label: {y_loaded[i]}, Encoded Feature Sample: {X_loaded[i][:10]}")

Loaded 2000 images with shape (64, 64)
Fitting PCA model...


Extracting hybrid features: 100%|█████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 720.07it/s]


Hybrid feature shape: (2000, 2336)
Feature vector normalized. Ready for QCNN input or classical classifier.
Hybrid encoded features saved to: hybrid_encoded_petimages_qcnn.npz
Label: 0, Encoded Feature Sample: [2.21976709 3.88969971 1.6704381  3.00981385 3.44670793 3.26147403
 2.99879303 3.70665639 3.51437865 2.77287502]
Label: 0, Encoded Feature Sample: [-0.36989287 -0.39360712 -0.39191396 -0.40107958 -0.39085865 -0.37762145
 -0.39222142 -0.39390295 -0.38721022 -0.38012319]
Label: 0, Encoded Feature Sample: [-0.36989287 -0.39360712 -0.39191396 -0.40107958 -0.39085865 -0.37762145
 -0.39222142 -0.39390295 -0.38721022 -0.38012319]
Label: 0, Encoded Feature Sample: [-0.36989287 -0.39360712 -0.39191396 -0.40107958 -0.39085865 -0.37762145
 -0.39222142 -0.39390295 -0.38721022 -0.38012319]
Label: 0, Encoded Feature Sample: [-0.36989287 -0.39360712 -0.39191396 -0.40107958 -0.39085865 -0.37762145
 -0.39222142 -0.39390295 -0.38721022 -0.38012319]
